In [2]:
import pandas as pd
import numpy as np

In [3]:
tarihler=pd.read_csv("data_date.txt")
etiketler=pd.read_csv("data_etiket.txt")
haberler=pd.read_csv("data_haber.txt",sep="\n")

In [4]:
veriler=pd.concat([tarihler,haberler,etiketler],axis=1)

In [5]:
veriler

,date,HABER,DURUM
0,20.06.2022,Investing.com - Kripto para piyasalarındaki gü...,artar
1,18.06.2022,Bitcoin 20.000 doların altına düştüInvesting.c...,artar
2,17.06.2022,Investing.com - Kripto para piyasalarındaki gü...,azalir
3,16.06.2022,Investing.com - Kripto para piyasalarındaki gü...,artar
4,15.06.2022,Investing.com - Kripto para piyasalarındaki gü...,azalir
...,...,...,...
1202,30.09.2018,Merkezi olmayan borsaya olan talep yüksektir. ...,azalir
1203,28.09.2018,Investing.com – Kripto paralar Perşembe günü k...,azalir
1204,27.09.2018,"Investing.com - Litecoin Perşembe günü %10,28 ...",azalir
1205,26.09.2018,İngiltere menşeli The Knowledge Academy kurulu...,artar


In [6]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
import jpype
from nltk.stem.porter import PorterStemmer


class Binary:
    def __init__(self):
        ZEMBEREK_PATH = 'zemberek-full_old.jar'
        startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))
        TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
        self.morphology = TurkishMorphology.createWithDefaults()
        nltk.download('stopwords')
        nltk.download('punkt')
    def kelime_temizlik(self,metin):
        yeniMetin=''
        for i in metin:
            if i not in string.punctuation:
                yeniMetin += i
        yorum=yeniMetin
        yorum=yorum.replace("0","").replace("1","").replace("2","").replace("3","").replace("4","").replace("5","").replace("6","").replace("7","").replace("8","").replace("9","")
        yorum=yorum.lower()
        yorum=yorum.split()
        ps=PorterStemmer()
        yorum=[ps.stem(kelime) for kelime in yorum if not kelime in set(stopwords.words('turkish'))]
        yorum= ' '.join(yorum)
        token = word_tokenize(yorum)
        return token

    def kok_temizlik(self,metin):
        token=self.kelime_temizlik(metin)
        lemma_word_top=''
        for i in range(len(token)):
            lemma_word = str(self.morphology.analyzeAndDisambiguate(str(token[i])).bestAnalysis()[0].getLemmas()[0])
            lemma_word_top=f'{lemma_word_top} {lemma_word}'
        file = open("ozel_kelimeler.txt","r",encoding="utf-8")
        kelimeler=file.read().split("\n")
        binary=[]
        for i in range(len(kelimeler)):
            #if metin.count(kelimeler[i])>1:
            #    print(kelimeler[i])
            binary.append(metin.count(kelimeler[i]))
        return binary


In [7]:
bn=Binary()
X=[]
for i in range (len(veriler)):
    X.append(bn.kok_temizlik(veriler.HABER[i]))
    print(i)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zetze\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zetze\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
btc=pd.read_csv("btc.csv")

In [9]:
for i in range(len(btc["Şimdi"])): btc["Şimdi"][i] = btc["Şimdi"][i].replace(".","").replace(",",".")

In [10]:
sonraki_kapanislar=[]
for i in range(len(btc["Şimdi"])):
    if i == 0:
        sonraki_kapanislar.append("20720.4")
    else:
        sonraki_kapanislar.append(btc["Şimdi"][i-1])

In [11]:
sonraki_kapanislar= np.asarray(sonraki_kapanislar).astype(np.float32)

In [12]:
hacimler=[]
kapanislar=[]
sonraki_kapanislar_=[]
for i in range (len(btc.Tarih)):
    for j in range(len(veriler.date)):
        if btc.Tarih[i]==veriler.date[j]:
            sonraki_kapanislar_.append(sonraki_kapanislar[i])
            hacimler.append(btc["Hac."][i])
            kapanislar.append(btc["Şimdi"][i])
            break
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
kapanislar=pd.DataFrame(kapanislar,columns=["kapanis"])
sonraki_kapanislar_=pd.DataFrame(sonraki_kapanislar_,columns=["yarin_kapanis"])

In [14]:
veriler=pd.concat([kapanislar,sonraki_kapanislar_,veriler],axis=1)

In [15]:
columns_titles = ["date","HABER","kapanis","yarin_kapanis","DURUM"]
veriler=veriler.reindex(columns=columns_titles)

In [16]:
veriler.iloc[:,0:4]

,date,HABER,kapanis,yarin_kapanis
0,20.06.2022,Investing.com - Kripto para piyasalarındaki gü...,20572.3,20720.400391
1,18.06.2022,Bitcoin 20.000 doların altına düştüInvesting.c...,18986.5,20577.199219
2,17.06.2022,Investing.com - Kripto para piyasalarındaki gü...,20444.6,18986.500000
3,16.06.2022,Investing.com - Kripto para piyasalarındaki gü...,20386.6,20444.599609
4,15.06.2022,Investing.com - Kripto para piyasalarındaki gü...,22577.9,20386.599609
...,...,...,...,...
1202,30.09.2018,Merkezi olmayan borsaya olan talep yüksektir. ...,6635.2,6608.200195
1203,28.09.2018,Investing.com – Kripto paralar Perşembe günü k...,6636.0,6603.899902
1204,27.09.2018,"Investing.com - Litecoin Perşembe günü %10,28 ...",6689.2,6636.000000
1205,26.09.2018,İngiltere menşeli The Knowledge Academy kurulu...,6468.1,6689.200195


In [17]:
for i in range(len(X)):
    X[i].append(veriler.kapanis[i])

In [18]:
y=veriler.yarin_kapanis

In [19]:
X = np.asarray(X).astype(np.float32)

In [26]:
#5999 - 0.628
#3664 - 477 
maes=[]
import random
for i in range(1000000):
    a=random.randint(0,10000)
    from sklearn.model_selection import train_test_split
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=a)
    from sklearn.ensemble import GradientBoostingRegressor
    gb = GradientBoostingRegressor(n_estimators=100) 
    gb.fit(X_train,y_train)
    predict=gb.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,predict)
    from sklearn.metrics import mean_absolute_error
    mae=mean_absolute_error(y_test,predict)
    pre_durum=[]
    for i in range(len(y_test)):
        if X_test[i][-1]<predict[i]:
            pre_durum.append("artar")
        else :
            pre_durum.append("azalir")
    real_durum=[]
    for j in range(len(y_test)):
        if X_test[j][-1]<list(y_test)[j]:
            real_durum.append("artar")
        else :
            real_durum.append("azalir")
    toplam=0
    for i in range(len(y_test)):
        if pre_durum[i]==real_durum[i]:
            toplam=toplam+1
    acc=toplam/len(y_test)
    maes.append(mae)
    print(acc,a,mae)

0.5041322314049587 4826 675.4137229253329
0.49586776859504134 8337 612.7865572447101
0.4669421487603306 8614 704.2289064229367
0.512396694214876 5061 700.5875728323142
0.5 8891 698.5171417236936
0.4628099173553719 9347 623.1114132613446
0.43388429752066116 6614 786.6150396546892
0.4793388429752066 439 752.9464297663118
0.4793388429752066 8933 690.7750942257516
0.5289256198347108 2734 716.9146689806831
0.4628099173553719 5475 724.8251381294994
0.4049586776859504 6814 816.5281648645824
0.5454545454545454 3868 596.2946056624577
0.512396694214876 9294 689.1273001827955
0.4669421487603306 6015 720.0843059805758
0.512396694214876 1831 612.0845629119603
0.4628099173553719 4597 758.4705232300121
0.4669421487603306 9266 723.1420927065668
0.5247933884297521 6264 718.6988127246385
0.5 5125 689.0927131821298
0.48760330578512395 3045 806.3424986893814
0.5330578512396694 780 756.1981253195535
0.5041322314049587 6103 745.0380129569751
0.4462809917355372 3569 735.7368060680869
0.5165289256198347 1532 

0.5702479338842975 6362 658.360196650083
0.47520661157024796 6556 727.8096245222
0.47107438016528924 6994 694.3932099845982
0.5 9226 697.3548970521116
0.49586776859504134 7322 773.6493009504521
0.512396694214876 880 665.6352600571932
0.49173553719008267 6160 602.2819516767
0.5082644628099173 7982 592.9580603657581
0.5330578512396694 971 747.8746279559208
0.4834710743801653 8116 677.7174535329445
0.5330578512396694 5770 753.3383827885286
0.5 5107 745.8906957353552
0.512396694214876 6709 686.2108509345247
0.4669421487603306 4864 677.2775103844366
0.5413223140495868 9188 695.2917151581792
0.5041322314049587 6912 630.6496199002963
0.49173553719008267 7963 661.9000921420943
0.512396694214876 467 744.0776999221981
0.48760330578512395 5390 650.201856536064
0.4669421487603306 8658 678.4990968983101
0.5454545454545454 308 569.9746185175038
0.48760330578512395 7525 717.1590713734112
0.5206611570247934 7663 694.3881232563119
0.4834710743801653 9005 669.1798426376572
0.5041322314049587 6141 678.38

0.5041322314049587 8937 563.6051341083205
0.4380165289256198 4890 747.0105277251834
0.4256198347107438 3386 747.907269103295
0.49586776859504134 8762 629.0544506577652
0.4834710743801653 2666 672.0216435206147
0.5413223140495868 1595 647.1301185600082
0.5702479338842975 9354 628.2175530579012
0.4793388429752066 1477 612.9912315739198
0.5206611570247934 5437 583.8650269526867
0.4628099173553719 2736 709.1564021566397
0.5041322314049587 9495 735.6765243696674
0.47107438016528924 1830 721.6345314041203
0.5371900826446281 2128 652.7597511480159
0.5289256198347108 4508 675.3166071606246
0.4669421487603306 486 659.7584711161843
0.5 8147 721.6973495463226
0.4834710743801653 9244 631.923231923964
0.512396694214876 1758 727.2494823472372
0.5165289256198347 5421 710.5282477111699
0.5082644628099173 1578 737.603676586144
0.4793388429752066 2673 808.6896882471633
0.5289256198347108 4999 725.6828158077554
0.45041322314049587 633 686.5127708657867
0.5041322314049587 2686 641.0064757686824
0.47933884

0.48760330578512395 4765 699.0289833702533
0.5 4991 600.9903121445942
0.48760330578512395 922 616.4724950785502
0.4628099173553719 8792 776.7285166463816
0.5 9265 698.4281860818969
0.49586776859504134 7517 710.7149027925707
0.5206611570247934 2212 818.4068927055305
0.512396694214876 659 734.1822859690506
0.48760330578512395 4375 676.4774570978173
0.5165289256198347 4725 627.0116292099005
0.5371900826446281 970 786.636805197422
0.47520661157024796 8990 751.2993336072103
0.47520661157024796 4949 683.0702276564215
0.5247933884297521 8875 657.9817166926721
0.5041322314049587 7281 724.1851873220039
0.5 1042 642.6903620095619
0.5082644628099173 9632 707.3527935553512
0.49586776859504134 2516 605.2624394890751
0.5165289256198347 7108 725.1103044938737
0.5165289256198347 9082 668.2373289429529
0.5041322314049587 2033 789.0545604401465
0.5247933884297521 4361 616.8741075527787
0.5082644628099173 6062 671.8543736779297
0.45454545454545453 7757 667.5750966366144
0.4669421487603306 3113 742.106831

0.5082644628099173 8409 777.5954418172566
0.5 1613 586.1057193538825
0.4793388429752066 5846 644.8864893569497
0.47520661157024796 2771 576.5243703067863
0.5041322314049587 891 634.6430551438294
0.48760330578512395 3047 755.3259898306304
0.5165289256198347 1558 617.5811102082017
0.4834710743801653 8892 671.948314648499
0.48760330578512395 3051 677.7066954573907
0.4628099173553719 6421 748.8453812534788
0.47520661157024796 7733 731.1978024670244
0.5165289256198347 1805 634.1258346605406
0.5165289256198347 2288 713.8521944774642
0.4462809917355372 7317 807.9384881768142
0.49173553719008267 2972 585.1042674083536
0.4380165289256198 8985 719.5300177132624
0.49173553719008267 6534 715.3701183955136
0.5289256198347108 8981 749.8590544203537
0.5082644628099173 9313 750.8968014137123
0.47520661157024796 5671 716.5587501022919
0.5082644628099173 9319 694.0566200364211
0.5495867768595041 8574 608.4503132168097
0.5165289256198347 6226 767.7046916461187
0.4669421487603306 9381 744.1149530145861
0.

...

0.5082644628099173 3860 594.1801823634871
0.45041322314049587 6698 763.0025081120981
0.5206611570247934 8879 722.263752760519
0.4628099173553719 6419 678.325560144268
0.49173553719008267 7013 614.3627744701341
0.5 5119 716.761535710716
0.5 5107 744.0146503930499
0.5082644628099173 517 639.8139721670728
0.4793388429752066 3703 810.8538975632222
0.5165289256198347 2507 737.9541263553238
0.5082644628099173 7032 788.6116121963779
0.49173553719008267 3606 586.7019383511982
0.5082644628099173 5408 697.4213047809652
0.48760330578512395 3958 726.0747373213894
0.5041322314049587 4755 689.9707130289559
0.5165289256198347 3241 661.5949988108885
0.5 5781 699.4859764648711
0.4834710743801653 3063 661.9375290226899
0.5082644628099173 3274 601.8508061644559
0.47520661157024796 9675 671.7354443740647
0.49173553719008267 9577 749.6530119702952
0.48760330578512395 2942 665.023959503317
0.4628099173553719 9148 726.6561491063352
0.49173553719008267 1931 664.9511328932681
0.47107438016528924 1102 703.22335

0.49173553719008267 3891 736.4510742522465
0.48760330578512395 8162 815.9225048960997
0.5330578512396694 4898 644.0022710382339
0.4256198347107438 194 750.7419992036135
0.49586776859504134 4517 825.0108785206406
0.5 2072 650.5429947533908
0.48760330578512395 1591 713.564655080493
0.5 1989 653.2376855079673
0.45041322314049587 9228 732.9756897598036
0.4380165289256198 4935 803.2410512129024
0.5619834710743802 3584 595.7740220698316
0.47107438016528924 5114 652.4075400069225
0.49586776859504134 8088 638.8549562958476
0.5289256198347108 7900 713.9732797029386
0.5537190082644629 1743 687.3180786701134
0.5 7175 680.334343030067
0.512396694214876 9208 779.4570343284894
0.5 2432 693.5469809984188
0.5041322314049587 8746 665.7174327790531
0.47107438016528924 2459 697.3003882863906
0.4793388429752066 7923 728.2754909922487
0.49586776859504134 932 702.1861978711364
0.47520661157024796 5215 621.6421446248855
0.45867768595041325 693 710.0552777650911
0.5330578512396694 4144 679.1504985776359
0.458

0.5413223140495868 1815 693.221373703802
0.49173553719008267 5146 689.7204234542355
0.4834710743801653 7490 643.544998233448
0.49586776859504134 5107 744.9965083834063
0.49173553719008267 2755 609.7021938892736
0.44214876033057854 6394 680.8330899634013
0.5289256198347108 6034 694.9925408330565
0.5041322314049587 762 726.5696181846321
0.49586776859504134 1056 729.4543079433681
0.4462809917355372 3656 695.9714801373514
0.49586776859504134 6906 661.4492775024771
0.5206611570247934 2936 647.7174972728648
0.43388429752066116 825 724.838545663347
0.5537190082644629 6779 644.9131910178032
0.5165289256198347 6304 629.6197800769187
0.4834710743801653 7969 658.1648245707885
0.5371900826446281 1905 663.1426489916635
0.5330578512396694 4362 614.5639698819217
0.49173553719008267 3090 653.1733775566763
0.47107438016528924 309 606.1962047483036
0.5041322314049587 7683 721.1976960378416
0.5 5781 700.0340954848257
0.5454545454545454 7767 713.1719753773913
0.4628099173553719 132 765.3755297822839
0.483

0.5 7428 648.6464801595064
0.512396694214876 2015 753.1202077442306
0.5041322314049587 2056 701.7046283730066
0.45867768595041325 8117 691.7466908051596
0.47107438016528924 9851 730.5048659680621
0.4793388429752066 7333 744.9384431364948
0.5661157024793388 8104 653.2101755111664
0.4669421487603306 7855 646.7864361316394
0.5 2982 668.1509147981155
0.5 8843 690.369626500541
0.45041322314049587 4806 863.3274823335571
0.512396694214876 2916 697.7548659179605
0.5413223140495868 8956 672.6135384881774
0.49173553719008267 3862 645.5759493267752
0.47107438016528924 6281 833.6332719433008
0.4628099173553719 9605 731.1348329853074
0.512396694214876 9693 760.8968364057116
0.49586776859504134 9503 754.666713042045
0.5247933884297521 7614 525.8789522258043
0.5289256198347108 3466 698.6131342850061
0.5 669 711.0705448999257
0.5247933884297521 3613 612.4318911342448
0.5041322314049587 4165 671.5698378615937
0.4462809917355372 318 645.769772106854
0.4793388429752066 8896 680.1074373456493
0.4958677685

KeyboardInterrupt: 

In [28]:
min(maes)

477.5169383362341

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=3664)
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=100) 
gb.fit(X_train,y_train)
predict=gb.predict(X_test)
from sklearn.metrics import r2_score
r2=r2_score(y_test,predict)
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test,predict)
print(mae)

475.86533683541865


In [22]:
real=pd.DataFrame(list(y_test),columns=["real"])
predict_df=pd.DataFrame(predict,columns=["predict"])

In [24]:
karsilastir=pd.concat([real,predict_df],axis=1)

In [26]:
karsilastir[0:20]

,real,predict
0,9655.700195,9692.226904
1,30097.800781,30307.151338
2,10672.900391,10259.394078
3,17730.699219,17255.388319
4,9631.200195,9672.228010
5,14019.900391,13424.307181
6,4004.100098,3912.835670
7,6468.899902,5726.989911
8,8000.399902,7975.701550
9,3990.399902,3984.535904
